In [1]:
# import the libraries as shown below
from tensorflow.keras.optimizers import RMSprop
from keras.layers import Input, Lambda, Dense, Flatten
from keras.layers import Input, Dense, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt                    

In [11]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

from sklearn.model_selection import train_test_split

from glob import glob
from skimage.io import imread
from os import listdir
import seaborn as sns
sns.set()

import time
import copy
from tqdm import tqdm_notebook as tqdm

import tensorflow as tf
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing import image
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import categorical_crossentropy
from keras.models import Sequential, Model
from keras.layers import Input, Conv2D, MaxPooling2D,GlobalAveragePooling2D
from keras.layers import Activation, Dropout, BatchNormalization, Flatten, Dense, AvgPool2D,MaxPool2D
import cv2

In [12]:
base_path = "C:/Users/umroot/Downloads/Dataset_ml/IDC_regular_ps50_idx5/"
folder = listdir(base_path)
len(folder)

279

In [13]:
total_images = 0
for n in range(len(folder)):
    patient_id = folder[n]
    for c in [0, 1]:
        patient_path = base_path + patient_id 
        class_path = patient_path + "/" + str(c) + "/"
        subfiles = listdir(class_path)
        total_images += len(subfiles)
        
total_images

277524

In [14]:
data = pd.DataFrame(index=np.arange(0, total_images), columns=["patient_id", "path", "target"])

k = 0
for n in range(len(folder)):
    patient_id = folder[n]
    patient_path = base_path + patient_id 
    for c in [0,1]:
        class_path = patient_path + "/" + str(c) + "/"
        subfiles = listdir(class_path)
        for m in range(len(subfiles)):
            image_path = subfiles[m]
            data.iloc[k]["path"] = class_path + image_path
            data.iloc[k]["target"] = c
            data.iloc[k]["patient_id"] = patient_id
            k += 1  

data.head()

,patient_id,path,target
0,10253,C:/Users/umroot/Downloads/Dataset_ml/IDC_regul...,0
1,10253,C:/Users/umroot/Downloads/Dataset_ml/IDC_regul...,0
2,10253,C:/Users/umroot/Downloads/Dataset_ml/IDC_regul...,0
3,10253,C:/Users/umroot/Downloads/Dataset_ml/IDC_regul...,0
4,10253,C:/Users/umroot/Downloads/Dataset_ml/IDC_regul...,0


In [15]:
data.loc[:, "target"] = data.target.astype(np.str)

In [16]:
patients = data.patient_id.unique()
patients = patients[:round(len(patients)*.2)]
sub_train_ids, test_ids = train_test_split(patients,
                                           test_size=0.2,
                                           random_state=0)
train_ids, valid_ids = train_test_split(sub_train_ids, test_size=0.1, random_state=0)

In [17]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 40, 
                                   width_shift_range = 0.2, 
                                   height_shift_range = 0.2, 
                                   shear_range = 0.2, 
                                   zoom_range = 0.2, 
                                   horizontal_flip = True, 
                                   vertical_flip =True)
test_datagen = ImageDataGenerator(rescale = 1./255)
valid_datagen = ImageDataGenerator(rescale = 1./255)

In [18]:
NUM_CLASSES = 2
target_size = (224,224)
batch_size = 32

In [21]:
train_df = data.loc[data.patient_id.isin(train_ids),:].copy()
test_df = data.loc[data.patient_id.isin(test_ids),:].copy()
valid_df = data.loc[data.patient_id.isin(valid_ids),:].copy()

print(f"train set shape: {train_df.shape}")
print(f"test set shape: {test_df.shape}")
print(f"validation set shape: {valid_df.shape}")

train set shape: (47697, 3)
test set shape: (14629, 3)
validation set shape: (7383, 3)


In [22]:
train_batches = train_datagen.flow_from_dataframe(
    train_df,
    x_col = 'path', 
    y_col ='target',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True,
)

valid_batches = valid_datagen.flow_from_dataframe(
    valid_df,
    x_col = 'path', 
    y_col ='target',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True,
)

test_batches = test_datagen.flow_from_dataframe(
    test_df,
    x_col = 'path', 
    y_col ='target',
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False,
)

Found 47697 validated image filenames belonging to 2 classes.
Found 7383 validated image filenames belonging to 2 classes.
Found 14629 validated image filenames belonging to 2 classes.


In [23]:
image_input = Input(shape=(224, 224, 3))
model = InceptionV3(input_tensor=image_input, include_top=True,weights='imagenet')

In [24]:
# don't train existing weights
for layer in model.layers[:-2]:
  layer.trainable = False
#model.summary()

In [33]:
last_layer = model.get_layer('avg_pool').output
x= Flatten(name='flatten')(last_layer)
out = Dense(1, activation='sigmoid', name='output')(last_layer)
model = Model(image_input, out)
custom_inception_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                           

In [34]:
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate= 0.0001), metrics=['accuracy'])
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    verbose=2)
model_checkpoints = tf.keras.callbacks.ModelCheckpoint("Breast_Cancer_inceptionv3_model.h5", save_best_only=True, verbose = 0)
callbacks = [
    early_stopping,
    model_checkpoints
]

In [ ]:
inception_history = model.fit(train_batches, validation_data = valid_batches, epochs = 100, 
                    callbacks = [callbacks], verbose = 1)

Epoch 1/100
  23/1491 [..............................] - ETA: 2:58:01 - loss: 0.6440 - accuracy: 0.7120